In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Gerekli kütüphaneleri içe aktarma
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
import kerastuner as kt


<ipython-input-2-db36f9c1ed2a>:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
import os
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))


['unripe', 'ripe', 'rotten', 'overripe']


In [4]:
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification/'
CATEGORIES = ["overripe", "ripe", "rotten", "unripe"]
IMG_SIZE = 100


In [5]:
def create_all_data():
    all_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                all_data.append([new_array, class_num])
            except Exception as e:
                pass
    return all_data

In [6]:
all_data = create_all_data()
print("Total number of images:", len(all_data))

Total number of images: 13478


In [7]:

X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [9]:
# Etiketleri kategorik hale dönüştürme
train_yCl = to_categorical(y_train, num_classes=4)
test_yCl = to_categorical(y_test, num_classes=4)
valid_yCl = to_categorical(y_val, num_classes=4)

In [10]:
print(y_test)

[1 1 3 ... 1 3 3]


In [11]:
def build_model(hp):
    model = Sequential()

    # İlk Conv2D katmanındaki birim sayısını ayarlama
    model.add(Conv2D(hp.Int('conv_1_units', min_value=32, max_value=128, step=32),
                     (3, 3), padding='same',
                     input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

   # İkinci Conv2D katmanındaki birim sayısını ayarlama
    model.add(Conv2D(hp.Int('conv_2_units', min_value=32, max_value=128, step=32),
                     (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    # Dense katmanındaki nöron sayısını ayarlama
    model.add(Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [12]:
# Hiperparametre ayarlayıcısını başlatma
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='banana_classification'
)



In [13]:
print(y_test)

[1 1 3 ... 1 3 3]


In [14]:
# Geri çağırmaları tanımma
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10)
]

# Hiperparametre ayarlamasını başlatma
tuner.search(x_train, train_yCl, batch_size=32, epochs=10, validation_data=(x_val, valid_yCl), callbacks=callbacks)


Trial 5 Complete [00h 01m 27s]
val_accuracy: 0.5291936993598938

Best val_accuracy So Far: 0.5959221720695496
Total elapsed time: 00h 05m 35s


In [18]:
# En iyi hiperparametreleri elde edme
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Modeli en iyi hiperparametrelerle oluşturma ve eğitme
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, train_yCl, epochs=50, validation_data=(x_val, valid_yCl))


Epoch 1/50
304/304 [==============================] - 6s 17ms/step - loss: 8.0003 - accuracy: 0.3380 - val_loss: 1.3083 - val_accuracy: 0.3976
Epoch 2/50
304/304 [==============================] - 5s 16ms/step - loss: 1.3178 - accuracy: 0.3806 - val_loss: 1.2839 - val_accuracy: 0.3957
Epoch 3/50
304/304 [==============================] - 5s 16ms/step - loss: 1.2500 - accuracy: 0.4296 - val_loss: 1.1992 - val_accuracy: 0.4606
Epoch 4/50
304/304 [==============================] - 5s 16ms/step - loss: 1.1774 - accuracy: 0.4710 - val_loss: 1.1859 - val_accuracy: 0.4791
Epoch 5/50
304/304 [==============================] - 5s 16ms/step - loss: 1.0977 - accuracy: 0.5204 - val_loss: 1.2247 - val_accuracy: 0.4847
Epoch 6/50
304/304 [==============================] - 5s 16ms/step - loss: 1.0082 - accuracy: 0.5723 - val_loss: 1.0286 - val_accuracy: 0.5755
Epoch 7/50
304/304 [==============================] - 5s 16ms/step - loss: 0.9150 - accuracy: 0.6154 - val_loss: 1.1153 - val_accuracy: 0.5032

In [19]:
# Modeli değerlendirme
print("Training Accuracy:", model.evaluate(x_train, train_yCl)[1])
print("Validation Accuracy:", model.evaluate(x_val, valid_yCl)[1])
print("Test Accuracy:", model.evaluate(x_test, test_yCl)[1])


304/304 [==============================] - 2s 5ms/step - loss: 0.0185 - accuracy: 0.9966
Training Accuracy: 0.996599018573761
34/34 [==============================] - 0s 5ms/step - loss: 1.3939 - accuracy: 0.8573
Validation Accuracy: 0.8572752475738525
85/85 [==============================] - 1s 6ms/step - loss: 1.2751 - accuracy: 0.8676
Test Accuracy: 0.8675816059112549


In [20]:
# Test seti üzerindeki tahminler
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = y_test

85/85 [==============================] - 0s 4ms/step


In [21]:
# Performans metrikler
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)


In [22]:
# Sonuçları görüntüleme
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Accuracy: 0.8675816023738873
Precision: 0.8717460354555036
Recall: 0.8675816023738873
F1 Score: 0.8676726863662322

Confusion Matrix:
 [[438  82  25   0]
 [ 11 725  49   6]
 [ 48  80 751  23]
 [  1  22  10 425]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.80      0.84       545
           1       0.80      0.92      0.85       791
           2       0.90      0.83      0.86       902
           3       0.94      0.93      0.93       458

    accuracy                           0.87      2696
   macro avg       0.88      0.87      0.87      2696
weighted avg       0.87      0.87      0.87      2696

